    model_count <- glm.fit(X, Y, family = poisson(), weights = weights, offset = offsetx)
    model_zero <- glm.fit(Z, as.integer(Y0), weights = weights, family = binomial(link = linkstr), offset = offsetz)
    start <- list(count = model_count$coefficients, zero = model_zero$coefficients)
    ## EM estimation of starting values
    if(control$EM & dist == "poisson") {
      mui <- model_count$fitted
      probi <- model_zero$fitted
      probi <- probi/(probi + (1-probi) * dpois(0, mui))
      probi[Y1] <- 0

      ll_new <- loglikfun(c(start$count, start$zero))
      ll_old <- 2 * ll_new
    
      while(abs((ll_old - ll_new)/ll_old) > control$reltol) {
        ll_old <- ll_new
        model_count <- glm.fit(X, Y, weights = weights * (1-probi), offset = offsetx,
	  family = poisson(), start = start$count)
        model_zero <- suppressWarnings(glm.fit(Z, probi, weights = weights, offset = offsetz,
	  family = binomial(link = linkstr), start = start$zero))
        mui <- model_count$fitted
        probi <- model_zero$fitted
        probi <- probi/(probi + (1-probi) * dpois(0, mui))
        probi[Y1] <- 0
        start <- list(count = model_count$coefficients, zero = model_zero$coefficients)
        ll_new <- loglikfun(c(start$count, start$zero))
      }
    }

In [34]:
loglikfun = ziPoisson
gradfun = gradPoisson
reltol = (np.finfo(float).eps)**(1/1.6)

In [35]:
print(reltol)

1.64636126996e-10


In [51]:
?sm.GLM

In [55]:

model_count = poisson_mod = sm.GLM(endog = Y, exog = X, family = sm.families.Poisson(),\
                                  offset = offsetx , freq_weights = weights).fit()
model_zero = sm.GLM(Y0.astype(int), exog = Z, family=sm.families.Binomial(), \
                   offset = offsetz , freq_weights = weights).fit()
start = {'zero':model_zero.params, 'count':model_count.params}

mui = model_count.predict()
probi = model_zero.predict()
probi = probi/(probi + (1-probi)*sp.stats.poisson.pmf(0, mui))
probi[Y1] = 0

ll_new = loglikfun(np.hstack((start['count'].values,start['zero'].values)))
ll_old = 2 * ll_new

while np.absolute((ll_old - ll_new)/ll_old) > reltol :
    ll_old = ll_new
    model_count = poisson_mod = sm.GLM(endog = Y, exog = X, family = sm.families.Poisson(),\
                                  offset = offsetx , freq_weights = weights*(1-probi), \
                                      start_params = start['count']).fit()
    model_zero = sm.GLM(Y0.astype(int), exog = Z, family=sm.families.Binomial(),\
                        offset = offsetz, freq_weights = weights, \
                        start_params = start['zero']).fit()
    start = {'zero':model_zero.params, 'count':model_count.params}

    mui = model_count.predict()
    probi = model_zero.predict()
    probi = probi/(probi + (1-probi)*sp.stats.poisson.pmf(0, mui))
    probi[Y1] = 0

    ll_new = loglikfun(np.hstack((start['count'].values,start['zero'].values)))
    
ll_new

-16292.541381627983

In [49]:
import numpy as np
import pandas as pd
import patsy
df = pd.read_csv('DebTrivedi.csv',index_col = [0])
sel = np.array([1, 6, 7, 8, 13, 15, 18])-1
df = df.iloc[:,sel]
# produce design matrices from R-style formula
X_formula = 'ofp ~ hosp + health + numchron + gender + school + privins'
y, X = patsy.dmatrices(X_formula, df, return_type='dataframe')
Z_formula = 'ofp ~ health'
Z1 = patsy.dmatrices(Z_formula, df, return_type='dataframe')[1]

In [5]:
## convenience variables
n = len(Y)
kx = X.shape[1] # Number of columns in X
kz = Z.shape[1]
Y0 = Y <= 0
Y1 = Y > 0

offsetx = np.zeros(n)
offsetz = np.zeros(n)
weights = np.ones(n)

In [23]:
import statsmodels.api as sm
import scipy as sp
linkinv = sp.special.expit
linkobj = sp.special.logit

In [33]:
print(np.finfo(float).eps)

2.22044604925e-16


In [18]:
start['count']

Intercept              1.028874
health[T.excellent]   -0.361993
health[T.poor]         0.248307
gender[T.male]        -0.112320
privins[T.yes]         0.201687
hosp                   0.164797
numchron               0.146639
school                 0.026143
dtype: float64

In [17]:
sm.Poisson(endog = Y, exog = X, start_params = start['count']).fit().params

Optimization terminated successfully.
         Current function value: 4.078895
         Iterations 1


Intercept              1.028874
health[T.excellent]   -0.361993
health[T.poor]         0.248307
gender[T.male]        -0.112320
privins[T.yes]         0.201687
hosp                   0.164797
numchron               0.146639
school                 0.026143
dtype: float64

In [20]:
mui

array([  5.65859172,   5.9611865 ,  13.72968619, ...,  10.27743623,
         4.68458323,   1.74117657])

In [21]:
def ziPoisson(parms):
    ## count mean
    mu = np.exp(np.dot(X,parms[np.arange(kx)]) + offsetx)
    ## binary mean
    phi = linkinv(np.dot(Z, parms[np.arange(kx,kx+kz)]) + offsetz)
    ## log-likelihood for y = 0 and y >= 1
    loglik0 = np.log( phi + np.exp( np.log(1-phi) - mu ) ) ## -mu = dpois(0, lambda = mu, log = TRUE)
    loglik1 = np.log(1-phi) + sp.stats.poisson.logpmf(Y, mu)
    ## collect and return
    loglik = np.dot(weights[Y0],loglik0[Y0])+np.dot(weights[Y1],loglik1[Y1])
    return loglik

In [28]:
def gradPoisson(parms):
    ## count mean
    eta = np.dot(X,parms[np.arange(kx)]) + offsetx
    mu = np.exp(eta)
    ## binary mean
    etaz = np.dot(Z, parms[np.arange(kx,kx+kz)]) + offsetz
    muz = linkinv(etaz)
    ## densities at 0
    clogdens0 = -mu
    dens0 = muz*(1-Y1.astype(float)) + np.exp(np.log(1 - muz) + clogdens0)
    ## working residuals  
    wres_count = np.where(Y1,Y-mu,-np.exp(-np.log(dens0) + 
                                          np.log(1 - muz) + clogdens0 + np.log(mu))) 
    link_etaz = np.exp(etaz)/(1+np.exp(etaz))**2
    wres_zero  = np.where(Y1,-1/(1-muz) * link_etaz, \
                          (link_etaz - np.exp(clogdens0) * link_etaz)/dens0)
    
    
    return (np.hstack((np.expand_dims(wres_count*weights,axis=1)*X, \
                np.expand_dims(wres_zero*weights,axis=1)*Z))).sum(axis=0)

In [25]:
parms = np.hstack((count.values,zero.values))

In [26]:
ziPoisson(parms)

-16582.015992208042

In [27]:
loglikfun(parms)

-16582.015992208042

In [29]:
loglikfun = gradPoisson

In [30]:
loglikfun(parms)

array([  3.01583524e+03,   2.06987205e+02,   4.71837390e+02,
         1.31490242e+03,   2.09412944e+03,   1.04395443e+03,
         4.04704433e+03,   2.98589710e+04,  -5.55443115e+01,
        -1.27690077e+01,  -1.30386892e+00])

In [31]:
ziPoisson(parms)

-16582.015992208042